# WEB SCRAPING TAPI MASIH GABISAAAA
Pengumpulan data dengan Web Scraping dari website rumah123.com

Referensi:
https://github.com/tugusav/Rumah123-Data-Analysis

Oleh:
- 18219006 Marcelino Feihan
- 18219014 Zarfa Naida Pratista
- 18219058 Afif Fahreza

Import semua library yang dibutuhkan

In [24]:
import bs4
from urllib.request import urlopen as uReq, Request
from bs4 import BeautifulSoup as soup
import time
from tqdm import tqdm

Kelas untuk data rumah

In [25]:
class HouseData:
    def __init__(self,property_title, property_agent,location,phone_number,property_type,land_area,building_area,price_idr,num_bathroom,num_bedroom,garage_capacity):
        self.property_title=property_title
        self.property_agent=property_agent
        self.location=location
        self.phone_number=phone_number
        self.property_type=property_type
        self.land_area=land_area
        self.building_area=building_area
        self.price_idr=price_idr
        self.num_bathroom=num_bathroom
        self.num_bedroom=num_bedroom
        self.garage_capacity=garage_capacity

    def printObject(self):
        print(f'''
            Title : {self.property_title}
            Agen: {self.property_agent}
            Location : {self.location}
            Phone number: {self.phone_number}
            Type : {self.property_type}
            LT : {self.land_area}
            LB : {self.building_area}
            price : {self.price_idr}
            KM: {self.num_bathroom}
            KT: {self.num_bedroom}
            Kapasitas Garasi Mobil: {self.garage_capacity}
            ''')

Fungsi untuk manipulasi data harga dari string menjadi integer

In [26]:
def convert_harga_to_int(str):
    # format "Rp. 700 Jt" atau "Rp. 35 M" atau "Rp. 355"
    str = str.replace(",", ".") # mengubah koma menjadi titik untuk kebutuhan lfoat
    str_split = str.split()

    if len(str_split)<=2:
        # harga hanya mengambil bentuk integer dari indeks ke 1
        harga = int(float(str_split[1])*1000000) #asumsi semua dalam satuan juta
        return harga
    else:
        # len dari listnya >=3 (ada Jt atau M)
        if str_split[2].lower() == 'jt':
            harga = int(float(str_split[1])*1000000)
            return harga
        elif str_split[2].lower() == 'm':
            harga = int(float(str_split[1])*1000000000)
            return harga

Fungsi web scraper yang membuat objek berdasarkan nilai yang didapat saat scraping

In [27]:
def scrapeweb(listOfHouse, firstpage, lastpage):
    
    i = firstpage
    while i < lastpage+1:
        try: 
            # declare the url for the website
            base_url_rumah123 = "https://www.rumah123.com/jual/rumah/" +"?page=" + str(i) # firstpage as page param
            header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}
            request = Request(base_url_rumah123, headers = header)
            uClient = uReq(request)
            html_page = uClient.read()
            uClient.close()
            soup_page = soup(html_page, "lxml")

            print(f'Scraping {base_url_rumah123} ...\n')
            property_list = soup_page.findAll('div', class_='sc-fubCfw irPtrP')

            for prop in tqdm(property_list):
                try:
                    # listing property infos
                    property_featured_title = prop.find('div', class_='ui-organisms-card-r123-featured__title-info truncate relative ui-col')
                    property_title = property_featured_title.find('span', class_='ui-atomic-text ui-atomic-text--styling-heading-6 ui-atomic-text--typeface-primary ui-atomic-link--children').text

                    # bathroom, bedroom, and garage capacity
                    property_featured_bottom = prop.find('div', class_='ui-organisms-card-r123-featured__bottom-row flex flex-row flex-wrap ui-row flex-align-end flex-justify-between')
                    # array of max 3 elements (km kt g)
                    property_element = property_featured_bottom.findAll('span', 'ui-atomic-text ui-atomic-text--styling-default ui-atomic-text--typeface-primary ui-atomic-link--children') 
                    km = 0 if len(property_element) < 1 else int(property_element[0].text)
                    kt = 0 if len(property_element) < 2 else int(property_element[1].text)
                    garasi = 0 if len(property_element) < 3 else int(property_element[2].text)

                    # info regarding land and building area
                    property_area = prop.find('div', class_= 'flex ui-molecules-list__divider-line--horizontal flex-align-center flex-row flex-wrap relative ui-molecules-list')
                    # LT = Luas tanah, LB = Luas bangunan
                    try:
                        property_LT = property_area.text.split()[3]
                    except IndexError:
                        property_LT = 0
                    try:
                        property_LB = property_area.text.split()[7]
                    except IndexError:
                        property_LB = 0

                    if (km != 0 and kt != 0) and (property_LT != 0 and property_LB != 0):
                        # keep going

                        # agent info
                        agent = prop.find('h2').text

                        # phone info
                        if prop.find('div', class_='ui-atomic-card box-shadow-r123 ui-organisms-card-r123-featured ui-organisms-card-r123-featured--premier') is None:
                            phone_number = prop.find('div', class_='ui-atomic-card box-shadow-r123 ui-organisms-card-r123-featured')['phonenumber']
                        else:
                            phone_number = prop.find('div', class_='ui-atomic-card box-shadow-r123 ui-organisms-card-r123-featured ui-organisms-card-r123-featured--premier')['phonenumber']    
                        
                        phone_number = "+" + phone_number

                        # property type info
                        property_type = prop.find('div', class_='relative ui-col-12').text

                        # location info
                        property_location = property_featured_title.find('span', class_='ui-atomic-text ui-atomic-text--styling-default ui-atomic-text--typeface-primary ui-atomic-link--children').text

                        # price info
                        property_price_card = prop.find('div', class_='flex flex-align-baseline ui-organisms-card-r123-featured__price-info')
                        property_price = convert_harga_to_int(property_price_card.find('a', class_='ui-atomic-link ui-atomic-link--styling-underline-none').text)
                        
                        # creating object
                        newHouse = HouseData(property_title, agent, property_location, phone_number, property_type, int(property_LT), int(property_LB), property_price, km, kt, garasi)
                        # newHouse.printObject()
                        listOfHouse.append(newHouse)
                        # wait 2 second
                        time.sleep(2)
                    else:
                        # we don't need bad data
                        pass
                
                except (AttributeError):
                    # listing other than house type
                    pass
                except Exception as e:
                    print(e)
            i+=1
            
        except Exception as e:
            print("\n")
            print('Error:', e)
            print("Error or Page finished")
            break
        
            
    print("\n")
    print(f"Scraping page from {firstpage} until {lastpage} has finished!")

Tes untuk fungsi scraper

In [28]:
test = []
scrapeweb(test, 1,1)

Scraping https://www.rumah123.com/jual/rumah/?page=1 ...



100%|██████████| 1/1 [00:00<?, ?it/s]



Scraping page from 1 until 1 has finished!


In [29]:
len(test)

0